# Loading Data and Setup

Here, we load the experiment data and convert the results we have into columns that we can use for analysis.

In [1]:
import pandas as pd
import json

In [3]:
with open("/home/dni138/mozilla_ai/data/flowjudge_run_0_messages_tools_hammerbench_results.json", "r") as f:
    first_run = json.load(f)

with open("/home/dni138/mozilla_ai/data/flowjudge_run_1_messages_tools_hammerbench_results.json", "r") as f:
    second_run = json.load(f)

with open("/home/dni138/mozilla_ai/data/flowjudge_run_2_messages_tools_hammerbench_results.json", "r") as f:
    third_run = json.load(f)

In [4]:
hammerbench = pd.read_csv("/home/dni138/mozilla_ai/data/hammerbench_singleturn.csv")

In [5]:
data_to_column = {
    "flowjudge_run_0_messages_tools_hammerbench": first_run,
    "flowjudge_run_1_messages_tools_hammerbench": second_run,
    "flowjudge_run_2_messages_tools_hammerbench": third_run
}
i = 0

for key, data in data_to_column.items():
    explanations = []
    scores = []
    for line in data[key]:
        json_line = json.loads(line)
        explanations.append(json_line["explanation"])
        scores.append(json_line["score"])
    hammerbench["explanation_run_{}".format(i)] = explanations
    hammerbench["score_run_{}".format(i)] = scores
    i += 1

In [9]:
hammerbench['gt_label'] = [True if label == "ST-Perfect" else False for label in hammerbench.label ]

In [12]:
hammerbench.to_csv("/home/dni138/mozilla_ai/data/function_call_experiment.csv")

# Analysis

Here, we look at two different metrics to evaluat how FlowJudge did as a function calling judge. First, we assess how stable FlowJudge during experimentation. We do this by passing the same data points three different times to FlowJudge (i.e. running the experiment three times), and then finding the `cohen_kappa_score` between the three runs pairwise. Second, for each experiment, we check the f1-score, precision, recall, and confusion matrix.

In [13]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score

In [4]:
hammerbench = pd.read_csv("/home/dni138/mozilla_ai/data/function_call_experiment.csv")

In [16]:
row = hammerbench[hammerbench['label'] == "ST-Perfect"].loc[[4418]]

In [17]:
row

,Unnamed: 0,label,messages,tools,explanation_run_0,score_run_0,explanation_run_1,score_run_1,explanation_run_2,score_run_2,gt_label
4418,4418,ST-Perfect,"[{""role"": ""user"", ""content"": ""I want to see ho...","[{""name"": ""VideoPlayback.ShortVideos.playShort...",The function call JSON provided does not meet ...,1,The function call provided in the output does ...,2,The function call JSON provided does not meet ...,1,True


In [11]:
list(row.explanation_run_0)[0]

'The function call JSON provided does not meet the criteria set out in the evaluation rubric. \n\n1. **Sufficient Parameters**: The function call JSON for "Navigation.TrafficViolations.viewViolationDetail" includes all necessary parameters (plate_number, city, time) to execute the function. This criterion is met.\n\n2. **Relevance**: The function call is relevant to the user\'s query, which is to check a specific traffic violation record. This criterion is met.\n\n3. **Existence in Tool List**: The function "Navigation.TrafficViolations.viewViolationDetail" does exist in the tool list provided. This criterion is met.\n\nHowever, the function call JSON is missing the specific details required for the function to execute properly. The user asked to see how a specific vehicle (plate number "Su E98765") handled a speeding record in Nanjing the day before yesterday. While the function call JSON includes the necessary parameters, it does not specify the exact time as "The day before yesterda

In [12]:
list(row.explanation_run_1)[0]

'The function call provided in the output does not meet the criteria specified in the evaluation rubric. \n\n1. **Sufficient Parameters**: The function call "Navigation.TrafficViolations.viewViolationDetail" has the required parameters (plate_number, city, time) to execute the function.\n2. **Relevance**: The function call is relevant to the user\'s query, which is to check the speeding record of a specific vehicle in a specific city on a specific date.\n3. **Existence in Tool List**: The function "Navigation.TrafficViolations.viewViolationDetail" exists in the tool list provided.\n\nHowever, the output does not include a tool list. The tool list is provided separately and is not part of the function call. Therefore, the function call meets all the criteria, but the absence of a tool list in the output makes it incomplete.\n\nBased on the evaluation criteria and scoring rubric, the function call meets all the necessary conditions, but the lack of a tool list in the output is a signific

In [13]:
list(row.explanation_run_2)[0]

'The function call JSON provided does not meet the criteria set out in the evaluation rubric. \n\n1. **Sufficient Parameters**: The function call JSON does not provide enough parameters to execute the function. The tool list does not specify any required parameters for the "Navigation.TrafficViolations.viewViolationDetail" function.\n\n2. **Relevance**: The function call is relevant to the user\'s ask. The user wants to see the speeding record of a specific vehicle in Nanjing.\n\n3. **Existence in Tool List**: The chosen function does exist in the tool list.\n\nHowever, since the function call does not provide sufficient parameters, it fails to meet all the criteria. Therefore, it cannot be considered successful based on the rubric.\n\nThe correct score, based on the evaluation criteria and scoring rubric, should be 1, as it meets the relevance and existence criteria but not the sufficient parameters criterion.'

## Cohen Kappa

In [14]:
cohen_kappa_score(hammerbench.score_run_0, hammerbench.score_run_1)

0.26344492678403253

In [15]:
cohen_kappa_score(hammerbench.score_run_1, hammerbench.score_run_2)

0.27004185295065264

In [16]:
cohen_kappa_score(hammerbench.score_run_0, hammerbench.score_run_2)

0.27486275759596934

## Performance Metrics

In [19]:
hammerbench.columns

Index(['label', 'messages', 'tools', 'explanation_run_0', 'score_run_0',
       'explanation_run_1', 'score_run_1', 'explanation_run_2', 'score_run_2',
       'gt_label'],
      dtype='object')

In [37]:
score_to_bool = {
    0: False,
    1: False,
    2: True
}

In [40]:
hammerbench["score_run_0"].map(score_to_bool).fillna(False)

/tmp/ipykernel_25142/2427440671.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hammerbench["score_run_0"].map(score_to_bool).fillna(False)


0        False
1        False
2        False
3        False
4        False
         ...  
13049    False
13050    False
13051    False
13052    False
13053    False
Name: score_run_0, Length: 13054, dtype: bool

In [43]:
for i in range(3):
    print("----------RUN_{}-----------".format(i))
    print("F1 Score: {}".format(f1_score(hammerbench.gt_label, hammerbench["score_run_{}".format(i)].map(score_to_bool).fillna(False), average="macro", labels=[False, True])))
    print("Precision: {}".format(precision_score(hammerbench.gt_label, hammerbench["score_run_{}".format(i)].map(score_to_bool).fillna(False), average="macro", labels=[False, True])))
    print("Recall: {}".format(recall_score(hammerbench.gt_label, hammerbench["score_run_{}".format(i)].map(score_to_bool).fillna(False), average="macro", labels=[False, True])))
    print("Confusion Matrix: \n\n {} \n".format(confusion_matrix(hammerbench.gt_label, hammerbench["score_run_{}".format(i)].map(score_to_bool).fillna(False), labels=[False, True])))

----------RUN_0-----------
F1 Score: 0.5035039230266493
Precision: 0.6131466994997525
Recall: 0.5196778041969499
Confusion Matrix: 

 [[10743   195]
 [ 1995   121]] 

----------RUN_1-----------
F1 Score: 0.5046584311020123
Precision: 0.6251617793110659
Recall: 0.5206912064252164
Confusion Matrix: 

 [[10760   178]
 [ 1994   122]] 

----------RUN_2-----------
F1 Score: 0.49889597981284217
Precision: 0.6054650923850642
Recall: 0.5172079630126981
Confusion Matrix: 

 [[10751   187]
 [ 2007   109]] 



/tmp/ipykernel_25142/741388246.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  print("F1 Score: {}".format(f1_score(hammerbench.gt_label, hammerbench["score_run_{}".format(i)].map(score_to_bool).fillna(False), average="macro", labels=[False, True])))
/tmp/ipykernel_25142/741388246.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  print("Precision: {}".format(precision_score(hammerbench.gt_label, hammerbench["score_run_{}".format(i)].map(score_to_bool).fillna(False), average="macro", labels=[False, True])))
/tmp/ipykernel_25142/741388246.py:5: Futur